# Generate Test Questions

In [20]:
from training.training_data_generator import TrainingDataGenerator

# Initialize training data generator
generator = TrainingDataGenerator(
    question_bank_path='training/question_bank.md', 
    trees_csv_path='data/trees_df.csv', 
    patches_csv_path='data/patches_df.csv'
)

# Generate training data
df = generator.generate_dataset()
print(f"Generated {len(df)} training examples")
print(f"\n{df['intent'].value_counts()}")

# Extract test questions
test_questions = df['text']

Generated 506 training examples

intent
patch_requirements      108
tree_requirements       104
tree_recommendations     91
payment                  78
growth_time              65
transportation           60
Name: count, dtype: int64


# Test Intent Classifier

In [21]:
from source.intent_classifier import IntentClassifier

# Initialize intent classifier
classifier = IntentClassifier()

# Predict intent for each question
for question in test_questions:
    intent, confidence = classifier.predict(question)
    print(f'Question: {question}')
    print(f'Intent: {intent}')
    print(f'Confidence: {confidence}')
    print('')

Question: What level do I need to plant Orange?
Intent: tree_requirements
Confidence: 0.7929103228731019

Question: What are the requirements for the Taverley patch?
Intent: patch_requirements
Confidence: 0.9296773592188712

Question: What is the nearest teleport to the Catherby patch?
Intent: transportation
Confidence: 0.7739794388016246

Question: What should I plant at level 81?
Intent: tree_recommendations
Confidence: 0.8028747083780615

Question: What is the fastest way to the Kastori patch?
Intent: transportation
Confidence: 0.7585671244116468

Question: What tree should I plant at level 51?
Intent: tree_recommendations
Confidence: 0.8838936876033475

Question: What is the protection payment for Orange?
Intent: payment
Confidence: 0.7425346282086581

Question: What level do I need for Curry trees?
Intent: tree_requirements
Confidence: 0.8132554029153198

Question: What is the nearest teleport to the Nemus Retreat patch?
Intent: transportation
Confidence: 0.7391706302573557

Quest

# Test Answer Retriever

In [22]:
from source.intent_classifier import IntentClassifier
from source.answer_retriever import AnswerRetriever

# Initialize intent classifier
classifier = IntentClassifier()

# Initialize answer retriever
retriever = AnswerRetriever()

# Predict intent for each question
for question in test_questions:
    intent, confidence = classifier.predict(question)
    result = retriever.get_answer(question, intent)
    print(f'Question: {question}')
    print(f'Lookup: {result["lookup_value"]}')
    print(f'Intent: {intent}')
    if intent == 'tree_recommendations':
        print(f'Regular Tree: {result["regular_tree"]}')
        print(f'Fruit Tree: {result["fruit_tree"]}')
    else:
        print(f'Answer: {result["answer_value"]}')
    print('')

Question: What level do I need to plant Orange?
Lookup: Orange
Intent: tree_requirements
Answer: 39

Question: What are the requirements for the Taverley patch?
Lookup: Taverley
Intent: patch_requirements
Answer: nan

Question: What is the nearest teleport to the Catherby patch?
Lookup: Catherby
Intent: transportation
Answer: Catherby teleport, Gnome glider to White Wolf Mountain and run south, Camelot teleport

Question: What should I plant at level 81?
Lookup: 81
Intent: tree_recommendations
Regular Tree: Magic
Fruit Tree: Dragonfruit

Question: What is the fastest way to the Kastori patch?
Lookup: Kastori
Intent: transportation
Answer: Pendant of Ates teleport and run southwest, Quetzal to Kastori and run north

Question: What tree should I plant at level 51?
Lookup: 51
Intent: tree_recommendations
Regular Tree: Maple
Fruit Tree: Pineapple

Question: What is the protection payment for Orange?
Lookup: Orange
Intent: payment
Answer: 3 baskets of strawberries

Question: What level do I

# Test Response Generator

In [23]:
from source.intent_classifier import IntentClassifier
from source.answer_retriever import AnswerRetriever
from source.response_generator import ResponseGenerator

# Initialize intent classifier
classifier = IntentClassifier()

# Initialize answer retriever
retriever = AnswerRetriever()

# Initialize response generator
generator = ResponseGenerator()

# Predict intent for each question
for question in test_questions:
    intent, confidence = classifier.predict(question)
    result = retriever.get_answer(question, intent)
    response = generator.generate_response(intent, result)
    print(f'Question: {question}')
    print(f'Lookup: {result["lookup_value"]}')
    print(f'Intent: {intent}')
    if intent == 'tree_recommendations':
        print(f'Regular Tree: {result["regular_tree"]}')
        print(f'Fruit Tree: {result["fruit_tree"]}')
    else:
        print(f'Answer: {result["answer_value"]}')
    print(f'Response: {response}')
    print('')

Question: What level do I need to plant Orange?
Lookup: Orange
Intent: tree_requirements
Answer: 39
Response: Orange trees require level 39 Farming to plant.

Question: What are the requirements for the Taverley patch?
Lookup: Taverley
Intent: patch_requirements
Answer: nan
Response: The Taverley patch has no requirements.

Question: What is the nearest teleport to the Catherby patch?
Lookup: Catherby
Intent: transportation
Answer: Catherby teleport, Gnome glider to White Wolf Mountain and run south, Camelot teleport
Response: The Catherby fruit tree patch can be found on the east side of the beach.
To get there, you have a few options:
• Catherby teleport
• Gnome glider to White Wolf Mountain and run south
• Camelot teleport

Question: What should I plant at level 81?
Lookup: 81
Intent: tree_recommendations
Regular Tree: Magic
Fruit Tree: Dragonfruit
Response: The best trees to grow at level 81 Farming are Magic and Dragonfruit.

Question: What is the fastest way to the Kastori patch?

# Test Patch Type Disambiguation

In [24]:
question = 'How do I get to the gnome stronghold patch?'
intent, confidence = classifier.predict(question)
result = retriever.get_answer(question, intent)
response = generator.generate_response(intent, result)
print(f'Question: {question}')
print(f'Intent: {intent}')
print(f'Lookup: {result["lookup_value"]}')
print(f'Answer: {result["answer_value"]}')
print(f'Response: {response}')
print('')

Question: How do I get to the gnome stronghold patch?
Intent: transportation
Lookup: Gnome Stronghold
Answer: Spirit Tree teleport
Response: The Gnome Stronghold patch can be found southwest of the Spirit Tree.
To get there, use a Spirit Tree teleport.



In [25]:
question = 'How do I get to the gnome stronghold fruit tree patch?'
intent, confidence = classifier.predict(question)
result = retriever.get_answer(question, intent)
response = generator.generate_response(intent, result)
print(f'Question: {question}')
print(f'Intent: {intent}')
print(f'Lookup: {result["lookup_value"]}')
print(f'Answer: {result["answer_value"]}')
print(f'Response: {response}')
print('')

Question: How do I get to the gnome stronghold fruit tree patch?
Intent: transportation
Lookup: Gnome Stronghold
Answer: Spirit Tree teleport
Response: The Gnome Stronghold fruit tree patch can be found east of the Spirit Tree.
To get there, use a Spirit Tree teleport.



In [26]:
question = 'What do I need to access the farming guild?'
intent, confidence = classifier.predict(question)
result = retriever.get_answer(question, intent)
response = generator.generate_response(intent, result)
print(f'Question: {question}')
print(f'Intent: {intent}')
print(f'Lookup: {result["lookup_value"]}')
print(f'Answer: {result["answer_value"]}')
print(f'Response: {response}')
print('')

Question: What do I need to access the farming guild?
Intent: patch_requirements
Lookup: Farming Guild
Answer: level 65 Farming
Response: Using the Farming Guild patch requires level 65 Farming.



In [27]:
question = 'What do I need to access the fruit tree patch in the farming guild?'
intent, confidence = classifier.predict(question)
result = retriever.get_answer(question, intent)
response = generator.generate_response(intent, result)
print(f'Question: {question}')
print(f'Intent: {intent}')
print(f'Lookup: {result["lookup_value"]}')
print(f'Answer: {result["answer_value"]}')
print(f'Response: {response}')
print('')

Question: What do I need to access the fruit tree patch in the farming guild?
Intent: patch_requirements
Lookup: Farming Guild
Answer: level 85 Farming
Response: Using the Farming Guild fruit tree patch requires level 85 Farming.

